<a href="https://colab.research.google.com/github/5G-Latency-Prediction-Project/5G-Data-Measurements/blob/main/ep5g_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import re
import os
from getpass import getpass

In [12]:
with open('./students-project-fall2023-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········




```
# Als Code formatiert
```

Install required packages and dependencies. Ignore the warnings.

In [20]:
!pip install loguru
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


Import packages

In [4]:
import json
import time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

In [13]:
leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

[
    {
        "name": "ep5g-lease",
        "id": "16dcd7fd-8b91-4f4c-bf4b-7466fd343a63",
        "reservation_id": "166dabc9-6fb4-4518-a65c-1ac1a6fb4aca",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:29:00.000000"
    },
    {
        "name": "adv-03-lease",
        "id": "21d16889-27db-434f-a2ae-1a9d2cf07d59",
        "reservation_id": "257e802b-6780-47ca-9c44-3e1dd6da95f6",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:30:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "aaa95f32-9e6c-4c9a-978a-f541765f3347",
        "reservation_id": "38be06d4-0cac-409d-aeb3-0f993cf6b08f",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:32:00.000000"
    },
    {
        "name": "worker-07-lease",
        "id": "f0e6b551-f979-44ab-94c1-36c06443275d",
        "reservation_id": "65bdde56-3c96-422b-81db-cc5bf941a079",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:31:00.000000"
    }
]


In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [12]:
# ep5g reservation
ep5g_lease = reserve(
    { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": "100", "duration": { "days":7, "hours":0 } }
)

# advantech router reservation
adv3_lease = reserve(
    { "type":"network", "name": "adv-03", "net_name": "adv-03", "segment_id": "133", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker7_lease = reserve(
    { "type":"device", "name":"worker-07", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker8_lease = reserve(
    { "type":"device", "name":"worker-08", "duration": { "days":7, "hours":0 } }
)

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-11-08 11:29:06.050 | INFO     | chi.expeca:reserve:167 - reserving ep5g
2023-11-08 11:29:09.105 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 to become "ACTIVE"
2023-11-08 11:29:14.257 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 is PENDING.
2023-11-08 11:29:19.420 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 is PENDING.
2023-11-08 11:29:24.564 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 is PENDING.
2023-11-08 11:29:29.726 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 is PENDING.
2023-11-08 11:29:34.882 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 16dcd7fd-8b91-4f4c-bf4b-7466fd343a63 is P

[
    {
        "name": "ep5g-lease",
        "id": "16dcd7fd-8b91-4f4c-bf4b-7466fd343a63",
        "reservation_id": "166dabc9-6fb4-4518-a65c-1ac1a6fb4aca",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:29:00.000000"
    },
    {
        "name": "adv-03-lease",
        "id": "21d16889-27db-434f-a2ae-1a9d2cf07d59",
        "reservation_id": "257e802b-6780-47ca-9c44-3e1dd6da95f6",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:30:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "aaa95f32-9e6c-4c9a-978a-f541765f3347",
        "reservation_id": "38be06d4-0cac-409d-aeb3-0f993cf6b08f",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:32:00.000000"
    },
    {
        "name": "worker-07-lease",
        "id": "f0e6b551-f979-44ab-94c1-36c06443275d",
        "reservation_id": "65bdde56-3c96-422b-81db-cc5bf941a079",
        "status": "ACTIVE",
        "end_date": "2023-11-15T11:31:00.000000"
    }
]


In [6]:
leaseslist = list_reservations(brief=True)

adv3_lease = [lease for lease in leaseslist if lease["name"]=="adv-03-lease"][0]
worker7_lease = [lease for lease in leaseslist if lease["name"]=="worker-07-lease"][0]
worker8_lease = [lease for lease in leaseslist if lease["name"]=="worker-08-lease"][0]

ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
worker7_reservation_id = worker7_lease["reservation_id"]
worker8_reservation_id = worker8_lease["reservation_id"]

print(adv3_lease,ep5g_lease, worker7_reservation_id, worker8_reservation_id)

{'name': 'adv-03-lease', 'id': '21d16889-27db-434f-a2ae-1a9d2cf07d59', 'reservation_id': '257e802b-6780-47ca-9c44-3e1dd6da95f6', 'status': 'ACTIVE', 'end_date': '2023-11-15T11:30:00.000000'} {'name': 'ep5g-lease', 'id': '16dcd7fd-8b91-4f4c-bf4b-7466fd343a63', 'reservation_id': '166dabc9-6fb4-4518-a65c-1ac1a6fb4aca', 'status': 'ACTIVE', 'end_date': '2023-11-15T11:29:00.000000'} 65bdde56-3c96-422b-81db-cc5bf941a079 38be06d4-0cac-409d-aeb3-0f993cf6b08f


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).
It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [15]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2023-11-09 14:51:54.277 | SUCCESS  | __main__:<cell line: 4>:4 - edge-net is created.
2023-11-09 14:51:56.702 | SUCCESS  | __main__:<cell line: 8>:8 - ep5g-vip-router router is created.
2023-11-09 14:51:56.705 | INFO     | __main__:<cell line: 9>:9 - {
    "id": "11c837d9-a381-4439-963e-17eea9b0ebaf",
    "name": "ep5g-vip-router",
    "tenant_id": "ddf790cc1bb64e0387eff22c1ab19f94",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.121"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2023-11-09T14:51:54Z",
    "updated_at": "2023-11-09T14:51:55Z",
    "revision_num

Now the network is ready to run the workloads. We start by running the edge-node perf-meas container.

In [16]:
edgenet = chi.network.get_network("edge-net")
chi.container.create_container(
    name = "edge-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker8_reservation_id,
    environment = {"SERVER_DIR":"/mnt/volume/"},
    mounts = [
        {'source': 'edge-volume', 'destination': '/mnt/volume/'}
    ],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.70.70.3/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active("edge-node")
logger.success("created edge-node container.")

2023-11-09 14:52:50.551 | SUCCESS  | __main__:<cell line: 20>:20 - created edge-node container.


Next, we run the end-node perf-meas container.

In [17]:
advnet = chi.network.get_network("adv-03-net")
chi.container.create_container(
    name = "end-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker7_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active("end-node")
logger.success("created end-node container.")

2023-11-09 14:53:38.997 | SUCCESS  | __main__:<cell line: 17>:17 - created end-node container.


Run uplink bandwidth test

In [18]:
command = "iperf3 -c 10.70.70.3 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-09 14:53:45.227 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


Run downlink bandwidth test

In [19]:
command = "iperf3 -c 172.16.0.96 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-09 14:53:49.839 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

### Uplink measurements

In [21]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "2023-11-03-t1"
client_ip = "172.16.0.40"
server_ip = "10.70.70.3"
sleep_dur = 10
tripm = "oneway"
total_samples = 1e6
interval = "10" # in ms
offset = "0" # ms
length = 100 # in B
mult = 1 # multiple of packets
duration = 30 # in seconds
total_session_time = interval # in seconds
itnum = 1 #math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [22]:
# run measurements at end node
command = f"DOWNLINK=false EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}ms INTERVAL_OFFSET={offset}ms LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-09 15:14:13.570 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
 # make parquet
dev_name = "endnode01"
trip = "uplink"

command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/client /mnt/volume/{exp_name}/edge/server /mnt/volume/{exp_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:36:56.135 | INFO     | __main__:<module>:10 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=sdr-test-project AUTH_USERNAME=gourav AUTH_PASSWORD=gouravexpeca python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-03 15:41:19.388 | INFO     | __main__:<module>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Downlink Measurement

In [ ]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "tstmps-DL"
server_ip = "172.16.0.40"
client_ip = "10.70.70.3"
sleep_dur = 60
tripm = "oneway"
total_samples = 1e6
interval = 10000 # in us
length = 100 # in B
mult = 1 # multiple of packets
duration = 1667 # in seconds
total_session_time = interval # in seconds
itnum = 1#math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [ ]:
 ###### run measurements at edge node #####
command = f"DOWNLINK=true EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}us LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-04 20:52:39.887 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
dev_name = "endnode01"
trip = "downlink"
 # make parquet
command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/edge/client /mnt/volume/{exp_name}/{dev_name}/server /mnt/volume/{exp_name}/{dev_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-05 08:31:14.630 | INFO     | __main__:<cell line: 9>:9 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=sdr-test-project AUTH_USERNAME=gourav AUTH_PASSWORD=gouravexpeca python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-09-03 18:35:59.316 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Reset Setup

In [ ]:
status = get_container_status("edge-node")
if status:
    chi.container.destroy_container("edge-node")
    wait_until_container_removed("edge-node")


status = get_container_status("end-node")
if status:
    chi.container.destroy_container("end-node")
    wait_until_container_removed("end-node")

logger.info(f"stopped and removed all containers")

# find the router again
router = None
try:
    router = chi.network.get_router("ep5g-vip-router")
except Exception as ex:
    logger.info(f"could not find ep5g-vip-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info(f"could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the edge-net")

# remove the leases
# unreserve_byid(ep5g_lease["id"])
# unreserve_byid(adv_lease["id"])
# unreserve_byid(worker_lease["id"])

2023-11-09 14:50:28.278 | INFO     | __main__:<cell line: 12>:12 - stopped and removed all containers
2023-11-09 14:50:30.732 | SUCCESS  | __main__:<cell line: 21>:24 - removed all routers from router
2023-11-09 14:50:31.355 | INFO     | __main__:<cell line: 21>:28 - checking all 3 subnets.
2023-11-09 14:50:32.846 | SUCCESS  | __main__:<cell line: 21>:34 - removed all subnets from router
2023-11-09 14:50:34.006 | SUCCESS  | __main__:<cell line: 21>:37 - deleted the router
2023-11-09 14:50:34.402 | INFO     | __main__:<cell line: 40>:43 - could not find edge-net.


In [ ]:
unreserve_byid(ep5g_lease["id"])
unreserve_byid(adv3_lease["id"])
unreserve_byid(worker7_lease["id"])
unreserve_byid(worker8_lease["id"])


2023-11-03 10:41:22.639 | INFO     | chi.expeca:remove_lease:84 - Removing ep5g-lease reservation with id 13454d25-bfcd-41a0-b387-d28cc1d55880.
2023-11-03 10:41:23.683 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 13454d25-bfcd-41a0-b387-d28cc1d55880 to become "None"
2023-11-03 10:41:28.724 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 13454d25-bfcd-41a0-b387-d28cc1d55880 is None.
2023-11-03 10:41:28.725 | SUCCESS  | chi.expeca:try_to_remove:95 - done
2023-11-03 10:41:28.749 | INFO     | chi.expeca:remove_lease:84 - Removing adv-03-lease reservation with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9.
2023-11-03 10:41:41.714 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for adv-03-lease with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9 to become "None"
2023-11-03 10:41:46.756 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-03-lease with id bc3aac29-52cd-4ef7-85d3-5b0dc59c9ff9

In [ ]:
ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
unreserve_byid(ep5g_lease["id"])
print(ep5g_lease)

IndexError: ignored